
# Projeto ICMC Júnior
## Objetivo: Prever se um funcionário vai sair da empresa (attrition).

### Etapa 1: Limpeza da base de dados

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [3]:
# Leitura do arquivo csv
df = pd.read_csv('dados.csv')


#### Visão resumida do dataframe

In [4]:
# Verificação do tamanho da base de dados
df.shape

# Exibe os 5 primeiros funcionários da base de dados, apenas para contextualizar com a base
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1.0,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1.0,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1.0,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,NaN,4,Life Sciences,NaN,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,NaN,591,Research & Development,2.0,1,Medical,NaN,7,...,4,80,1,6,3,3,2,2,2,2


#### Tratamento de colunas com valores irrelevantes ou constantes

In [5]:
# Tratamento de colunas com valores irrelevantes
# Verifica (e remove) colunas que possuem valores constantes
for coluna in df.columns.tolist(): 
    if df[coluna].nunique() <= 1:
        print(f"removendo coluna '{coluna}' (valor constante)")
        df.drop(columns=[coluna], inplace=True)
# Remove manualmente o EmployeeNumber (id do funcionário na base; como usamos o índice do dataframe, não é necessário)
if 'EmployeeNumber' in df.columns:
    print("removendo coluna 'EmployeeNumber' (id do funcionário)")
    df.drop(columns=['EmployeeNumber'], inplace=True)


removendo coluna 'EmployeeCount' (valor constante)
removendo coluna 'Over18' (valor constante)
removendo coluna 'StandardHours' (valor constante)
removendo coluna 'EmployeeNumber' (id do funcionário)


#### Tratamento de colunas com valores nulos

In [6]:
# Antes de tratar os valores faltantes, definimos as seguintes funções referentes a verificação de outliers
# Para substituir os valores nulos, devemos ter cuidado para não introduzir viés (a partir da presença ou não de outliers) na base de dados, por isso precisamos dessas funções.

def calculate_lim_iqr(data_column):
    """Função responsável por calcular a amplitude interquartil de um Série (Coluna)"""
    Q1 = data_column.quantile(0.25) # 1° Quartil 
    Q3 = data_column.quantile(0.75) # 3° Quartil 
    
    
    IQR = Q3 - Q1 # amplitude interquartil
    
    limite_inferior = Q1 - (1.5 * IQR) # lim inferior do box-plot
    limite_superior = Q3 + (1.5 * IQR) # lim superior do box-plot
    
    return limite_inferior, limite_superior

def verify_outliers(column, print_dados = True):
    """Calcula limite inferior e superior da Série fornecida (coluna)"""
    lim_inf, lim_sup = calculate_lim_iqr(df[column])
    
    # Verifica se possui um dado menor que o limite inferior ou se há um dado maior que o limite superior
    if df[column].min() < lim_inf or df[column].max() > lim_sup:
        if print_dados:
            print(f"{column}: lim_inf: {lim_inf} | lim_sup:{lim_sup} | min:{df[column].min()} | max:{df[column].max()}")
        return True
    return False

In [7]:
# Verifica quantos valores nulos tem por colunas, ou seja, por categoria.
df.isnull().sum() 

Age                          0
Attrition                    0
BusinessTravel              70
DailyRate                    0
Department                   0
DistanceFromHome            47
Education                    0
EducationField               0
EnvironmentSatisfaction     53
Gender                      67
HourlyRate                  32
JobInvolvement              61
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                0
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
OverTime                    51
PercentSalaryHike            0
PerformanceRating           46
RelationshipSatisfaction     0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInCurrentRole           0
YearsSinceLastPromotion      0
YearsWithCurrManager         0
dtype: int64

In [8]:
# Verificando manualmente entre as colunas existentes, podemos listar as colunas que possuíam valores nulos e o respectivo tipo delas (listas abaixo).
# Dessa forma, podemos padronizar o tratamento de valores nulos para as colunas qualitativas: preenche-se com a moda;
# e para as colunas quantitativas contínuas: preenche-se com a mediana ou média (a depender do caso, por isso fazemos uma análise prévia sobre elas)


""" Tratamento de valores nulos sobre colunas qualitativas --------------------------------------------------- """
cols_qual_com_nulos = [
    "EnvironmentSatisfaction",
    "OverTime",
    "BusinessTravel",
    "Gender",
    "JobInvolvement",
    "PerformanceRating"
]

for i in cols_qual_com_nulos:
    print("Preenchendo coluna:", i, "com a moda:", df[i].mode()[0])
    df[i] = df[i].fillna(df[i].mode()[0])


""" Tratamento de valores nulos sobre colunas quantitativas --------------------------------------------------- """
cols_quant_com_nulos = [
    "DistanceFromHome",
    "HourlyRate"
]

def analise_col_quantitativa(df, column_name):
    """Função para análise descritiva de colunas quantitativas contínuas"""
    print(f"\nAnálise descritiva da coluna: {column_name}")
    lim_inf, lim_sup = calculate_lim_iqr(df[column_name]) 
    print(f"Limite inferior: {lim_inf}, limite superior: {lim_sup}")
    print(f"Há outliers? {'Sim' if verify_outliers(column_name, print_dados=False) else 'Não'}")
    print(f"Coeficiente de variação: {df[column_name].std() / df[column_name].mean()}")
    print(f"Média: { df[column_name].mean()}, Mediana: { df[column_name].median()} e Moda: { df[column_name].mode()[0]}")

for i in cols_quant_com_nulos:
    analise_col_quantitativa(df, i)

# Decisão manual baseada na análise acima:
df["DistanceFromHome"] = df["DistanceFromHome"].fillna(df["DistanceFromHome"].median()) # Com o alto coeficiente de variação, é válido usar a mediana ao invés da média
print(f"Preenchendo coluna: DistanceFromHome com a mediana: {df['DistanceFromHome'].median()}")
df["HourlyRate"] = df["HourlyRate"].fillna(df["HourlyRate"].mean()) # Com o baixo coeficiente de variação, é válido usar a média ao invés da mediana
print(f"Preenchendo coluna: HourlyRate com a média: {df['HourlyRate'].mean()}")

Preenchendo coluna: EnvironmentSatisfaction com a moda: 3.0
Preenchendo coluna: OverTime com a moda: No
Preenchendo coluna: BusinessTravel com a moda: Travel_Rarely
Preenchendo coluna: Gender com a moda: Male
Preenchendo coluna: JobInvolvement com a moda: 3.0
Preenchendo coluna: PerformanceRating com a moda: 3.0

Análise descritiva da coluna: DistanceFromHome
Limite inferior: -16.0, limite superior: 32.0
Há outliers? Não
Coeficiente de variação: 0.8852210503240954
Média: 9.188334504567814, Mediana: 7.0 e Moda: 2.0

Análise descritiva da coluna: HourlyRate
Limite inferior: -6.0, limite superior: 138.0
Há outliers? Não
Coeficiente de variação: 0.3093151741293963
Média: 65.90751043115438, Mediana: 66.0 e Moda: 66.0
Preenchendo coluna: DistanceFromHome com a mediana: 7.0
Preenchendo coluna: HourlyRate com a média: 65.90751043115438


In [9]:
# Verifica se possui funcionários duplicados
df.duplicated().sum() 

np.int64(0)

In [10]:
# Resumo estatístico das colunas do dataframe após a substituição dos valores nulos
df.describe()

# Verifica se ainda há valores nulos
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

#### Tratamento de assimetria/outliers

In [11]:
# Para os tratamentos referidos, definimos previamente funções auxiliares:

def trata_outliers_clipping(df, colunas):
    """Trata outliers "capando" (clipping) os valores nos limites IQR"""
    
    for coluna in colunas:
        # pega os limites
        lim_inf, lim_sup = calculate_lim_iqr(df[coluna])

        # conta quantos outliers existem antes
        num_outliers = df[(df[coluna] < lim_inf) | (df[coluna] > lim_sup)].shape[0]

        if num_outliers > 0:
            print(f"Tratando {num_outliers} outliers em '{coluna}' com clipping")
            # usa np.clip para "capar" os valores (um valor fora do limite é substituído pelo próprio limite)
            df[coluna] = np.clip(df[coluna], lim_inf, lim_sup)

    return df


def trata_assimetria_log(df, colunas):
    """Aplica transformação log1p em colunas específicas com cauda longa (assimétricas)"""
    
    for col in colunas:
        if col in df.columns:
            print(f"Transformando coluna '{col}' com log1p")
            # substitui a coluna original pela sua versão log
            df[col] = np.log1p(df[col])
            # renomeia para clareza
            df.rename(columns={col: f'{col}_log'}, inplace=True)
            
    return df

In [12]:
# Definimos as colunas quantitativas contínuas para tratamento de outliers e assimetria
# Obs.: essas colunas foram definidas manualmente após análise prévia dos dados, não necessariamente são todas assimétricas, mas potencialmente podem ser
colunas_assimetricas = ['MonthlyIncome', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion']
# Obs.: as colunas que terão assimetria tratada não são incluídas na lista de colunas para clipping (pois não faz sentido tratar outliers após a transformação logarítmica)
colunas_para_clipping = ['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike']

# Tratamento de assimetria
df = trata_assimetria_log(df, colunas_assimetricas)

# Tratamento de outliers
df = trata_outliers_clipping(df, colunas_para_clipping)

Transformando coluna 'MonthlyIncome' com log1p
Transformando coluna 'TotalWorkingYears' com log1p
Transformando coluna 'YearsAtCompany' com log1p
Transformando coluna 'YearsInCurrentRole' com log1p
Transformando coluna 'YearsSinceLastPromotion' com log1p
Tratando 52 outliers em 'NumCompaniesWorked' com clipping


#### Padronização de texto

In [13]:
# Por fim, padronizamos os textos presentes no dataframe com a seguinte função auxiliar:

def padroniza_texto(df):
    """padroniza colunas de texto (object): remove espaços extras e converte para minúsculas"""
    
    print("Padronizando colunas de texto (strip, lower)...")
    df_tratado = df.copy()
    
    # seleciona apenas as colunas do tipo 'object' (texto)
    colunas_texto = df_tratado.select_dtypes(include=['object']).columns
    
    for col in colunas_texto:
        df_tratado[col] = df_tratado[col].str.strip().str.lower()
        
    return df_tratado

df = padroniza_texto(df)

Padronizando colunas de texto (strip, lower)...


#### Registro da base limpa

In [14]:
# Por fim, salvamos o dataframe tratado em um novo arquivo csv 

def salvar_arquivo(df, nome_csv):
    """Salva o dataframe em um arquivo csv"""
    print("Salvando arquivo...")
    if df is not None:
        try:
            df.to_csv(nome_csv, index=False, encoding='utf-8')
            print(f"Arquivo salvo com sucesso em: {nome_csv}")
        except Exception as e:
            print(f"ERRO ao salvar o arquivo: {e}")

salvar_arquivo(df, 'dados_limpos.csv')

Salvando arquivo...
Arquivo salvo com sucesso em: dados_limpos.csv
